In [1]:
!pip install pandas

In [2]:
import torch, baukit
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import numpy as np
from baukit import pbar
import os

/share/u/koyena/.conda/envs/rome/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /share/u/koyena/.conda/envs/rome/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [3]:
#MODEL_NAME = "gpt2-xl"  # gpt2-xl or EleutherAI/gpt-j-6B
MODEL_NAME = "gpt2-xl"
model, tok = (
    AutoModelForCausalLM.from_pretrained(MODEL_NAME, low_cpu_mem_usage=False).to("cuda"),
    AutoTokenizer.from_pretrained(MODEL_NAME)
)
baukit.set_requires_grad(False, model)

In [4]:
import baukit
baukit.set_requires_grad(False, model)

In [5]:
def get_key_vectors(model, tok, tokens, layer_names = None):
    import re
    from baukit import TraceDict
    inp = {k: torch.tensor(v)[None].cuda() for k, v in tok(tokens).items()}
    if layer_names is None:
        layer_names = [n for n,_ in model.named_modules() if 'mlp.c_proj' in n]
    with TraceDict(model, layer_names, retain_input=True) as tr:
        logits = model(**inp)['logits']
    return np.stack([tr[layername].input[0][-1].cpu().detach().numpy() for layername in layer_names])

tokens = 'emma watson'
hs = get_key_vectors(model, tok, tokens)
hs.shape
type(hs)

numpy.ndarray

In [6]:
#token_csv = "../data/tokens.csv"
#token_csv = "../data/country_cap_tokens.csv"
token_csv = "../data/custom_tokens.csv"
token_df = pd.read_csv(token_csv)
tokens = token_df['token'].values
token_types = token_df['token_type'].values

In [7]:
#npy_datapath = "../data/npy_files"
#npy_datapath = "../data/npy_files_2_token_cap"
npy_datapath = "../data/npy_files_custom"
mlp_proj_layers = [n for n,_ in model.named_modules() if 'mlp.c_proj' in n]
npy_filepaths = [npy_datapath + "/" + "-".join(n.split('.')[:3]) + ".npy"
                 for n in mlp_proj_layers]

In [8]:
def is_n_token_len(tok, tokens, n_len = None):
    inp = {k: torch.tensor(v)[None].cuda() for k, v in tok(tokens).items()}
    return len(inp['input_ids'][0]) == n_len

In [10]:
# only 2-token words:
if not os.path.exists(npy_datapath):
    os.mkdir(npy_datapath)
    
new_data_list = []
for i in range(len(tokens)):
    is_2_len = is_n_token_len(tok, tokens[i], 2)
    if is_2_len:
        new_data_list.append([tokens[i], token_types[i]])

print(len(new_data_list))
# new_data_df = pd.DataFrame(new_data_list, columns = ['token','token_type'])
# new_output_csv = "../data/2-len-cap-tokens.csv"
# new_data_df.to_csv(new_output_csv, index=False)        


# for npy_file, layer_name in pbar(zip(npy_filepaths, mlp_proj_layers)):
#     final_data = []
#     for i in range(len(new_data_list)):
#         data = get_key_vectors(model,tok,new_data_list[i][0],[layer_name])
#         if data is not None:
#             data = data.reshape(-1)
#             final_data.append(data)
#     final_data = np.array(final_data)
#     np.save(npy_file, final_data)

128


In [8]:
if not os.path.exists(npy_datapath):
    os.mkdir(npy_datapath)
for npy_file, layer_name in pbar(zip(npy_filepaths, mlp_proj_layers)):
    final_data = []
    for i in range(len(tokens)):
        data = get_key_vectors(model,tok,tokens[i],[layer_name])
        data = data.reshape(-1)
        final_data.append(data)
    final_data = np.array(final_data)
    np.save(npy_file, final_data)

0it [00:00, ?it/s]